In [ ]:
# imports
import cv2 as cv
import numpy as np
import time
from matplotlib import pyplot as plt
import glob
from prettytable import PrettyTable

# Time Measurement

In [ ]:
time_measurements = {}

def start_time_measurement(eventName):
    add = False
    if eventName not in time_measurements:
        time_measurements[eventName] = {
            "name": eventName,
            "start": [],
            "end": [],
            "count": 0
        }
        add = True
    elif len(time_measurements[eventName]["start"]) > len(time_measurements[eventName]["end"]):
        print(f"Time measure error: Event '{eventName}' not finished before reassignment!")
    else:
       add = True
    
    #start measurement as late as possible
    if add:
        time_measurements[eventName]["start"].append(time.perf_counter_ns())

def end_time_measurement(eventName):
    #end measurement as fast as possible
    temp_time = time.perf_counter_ns()
    if eventName not in time_measurements:
        print(f"Time measure error: Event '{eventName}' not defined!")
    elif len(time_measurements[eventName]["end"]) >= len(time_measurements[eventName]["start"]):
        print(f"Time measure error: Event '{eventName}' not started before reassignment!")
    else:
        time_measurements[eventName]["end"].append(temp_time)
        time_measurements[eventName]["count"] += 1

def analyse_time_measurements():
    time_measurements_table = PrettyTable(["Name", "Avg. [ms]", "Min. [ms]", "Max. [ms]", "Occurrences [compl.]"])
    time_measurements_table.align["Name"] = "l"
    time_measurements_table.align["Avg. [ms]"] = "r"
    time_measurements_table.align["Min. [ms]"] = "r"
    time_measurements_table.align["Max. [ms]"] = "r"
    time_measurements_table.align["Occurrences [compl.]"] = "r"
    for key, event in time_measurements.items():
        timings = []
        if len(event["start"]) != len(event["end"]):
            print(f"Time measure error: Event '{key}' has different amounts of values for start and end times!")
        else:
            #exclude 0 values
            for i in range(len(event["start"])):
                timing = (event["end"][i] - event["start"][i])
                if timing >= 0:
                    timing = timing / (1000 * 1000) #convert from ns to ms
                    timings.append(timing)

            event["min"] = min(timings) 
            event["max"] = max(timings)
            event["avg"] = sum(timings) / len(event["start"])

            time_measurements_table.add_row([key, '{0:.2f}'.format(event["avg"]), '{0:.2f}'.format(event["min"]), '{0:.2f}'.format(event["max"]), event["count"]])
    print(time_measurements_table)

# Kamerakalibrierung

In [ ]:
# size of chessboard, minimum error with (7, 6), but there were severe artefacts at the borders (see error calculation at the end)
x, y = 9, 6

# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((y * x, 3), np.float32)
objp[:, :2] = np.mgrid[0:x, 0:y].T.reshape(-1, 2)
# Arrays to store object points and image points from all the images.
objpoints = []  # 3d point in real world space
imgpoints = []  # 2d points in image plane.


images = glob.glob("./img/Udacity/calib/*.jpg")
for i, fname in enumerate(images):
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(gray, (x, y), None)
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        corners2 = cv.cornerSubPix(
            gray, corners, (11, 11), (-1, -1), criteria
        )  # improve accuracy of corners
        imgpoints.append(corners)

ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(
    objpoints, imgpoints, gray.shape[::-1], None, None
)

h, w = img.shape[:2]
newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w, h), 1, (w, h))


def undistort_image(img):
    h, w = img.shape[:2]

    img_undist = cv.undistort(img, mtx, dist, None, newcameramtx)
    # crop the image
    x, y, w, h = roi
    return img_undist[y : y + h, x : x + w]


# ~50% faster than undistort_image()
def undistort_image_remap(img):
    h, w = img.shape[:2]
    mapx, mapy = cv.initUndistortRectifyMap(mtx, dist, None, newcameramtx, (w, h), 5)
    dst = cv.remap(img, mapx, mapy, cv.INTER_LINEAR)
    # crop the image
    x, y, w, h = roi
    return dst[y : y + h, x : x + w]

# Perspektivtransformation

Mögliche Performance-verbesserung:

- Bild nach warp verkleinern, da ein großteil des Bildes aus wenigen Pixeln entsteht ->


In [ ]:
# udacity images
src_udacity = np.float32([[191, 628], [531, 404], [1021, 628], [681, 404]])
dst_udacity = np.float32([[150, 720], [150, 10], [1000, 720], [1000, 10]])
M_warp = cv.getPerspectiveTransform(src_udacity, dst_udacity)
M_rewarp = cv.getPerspectiveTransform(dst_udacity, src_udacity)


def warp_image_udacity(img):
    img = cv.warpPerspective(img, M_warp, (img.shape[1], img.shape[0]))
    # image = cv.resize(
    #     image, (int(img.shape[1] / 2), int(img.shape[0] / 2))
    # )  # resize to half size
    return img

def rewarp_image_udacity(img):
    # image = cv.resize(
    #     img, (int(img.shape[1] * 2), int(img.shape[0] * 2))
    # ) 
    # img = cv.warpPerspective(img, M_rewarp, (img.shape[1], img.shape[0]))
    img = cv.warpPerspective(img, M_rewarp, (img.shape[1], img.shape[0]), cv.WARP_INVERSE_MAP)
    return img

# Sliding Windows

In [ ]:
nwindows = 50
margin = 50
def sliding_windows(frame, window_width=200, minimum_whites=30, show_windows=False):
    # Histogram for image
    hist = np.sum(frame[frame.shape[0]//2:, :], axis=0)
        
    # Take peaks from left and right side of histogramm for starting points and add half margin
    mid = np.int32(hist.shape[0] // 2)
    leftx_start = np.argmax(hist[:mid]) - window_width // 2
    rightx_start = np.argmax(hist[mid:]) + mid + window_width // 2
    # Window height based on number of windows
    window_height = np.int32(frame.shape[0] // nwindows)
    
    # Calc points that are not zero in images
    nonzero = frame.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    # Initialize current positions for windows
    leftx_current = leftx_start
    rightx_current = rightx_start

    # Initialize values to be returned -> centers of windows
    lefts_good = np.empty(shape=(1,1), dtype=int)
    rights_good = np.empty(shape=(1,1), dtype=int)

    # Go through every window
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = frame.shape[0] - (window + 1) * window_height
        win_y_high = frame.shape[0] - window*window_height
        y_mid = (win_y_low + win_y_high) // 2
        
        # Calculate boundaries of the window
        win_xleft_low = leftx_current - window_width  
        win_xleft_high = leftx_current + window_width  
        win_xright_low =  rightx_current - window_width 
        win_xright_high = rightx_current + window_width  
        
        # Identify the pixels that are not zero within window
        left_inds = ((nonzeroy >= win_y_low ) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        right_inds = ((nonzeroy >= win_y_low ) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
        
        # If more than minimum pixels are found -> recenter next window
        if len(left_inds) > minimum_whites:
            leftx_current = np.int32(np.mean(nonzerox[left_inds]))
            lefts_good = np.append(lefts_good, leftx_current)
            if show_windows:
                cv.rectangle(frame, (leftx_current - margin, win_y_low),(leftx_current + margin, win_y_high),(255, 255, 255), 2)
        else:
            lefts_good = np.append(lefts_good, None)
        if len(right_inds) > minimum_whites:
            rightx_current = np.int32(np.mean(nonzerox[right_inds]))
            rights_good = np.append(rights_good, rightx_current)
            if show_windows:
                cv.rectangle(frame, (rightx_current - margin, win_y_low),(rightx_current + margin, win_y_high),(255, 255, 255), 2)
        else:
            rights_good = np.append(rights_good, None)

    return mid, lefts_good, rights_good

# PolyLines

In [ ]:
def drawPolyLines(frame, left_pts, right_pts):
    y = np.linspace(len(frame), 0, nwindows+1).astype(int)
    left_line = np.array(list(zip(left_pts[1:], y)))
    right_line = np.array(list(zip(right_pts[1:], y)))

    cv.polylines(frame, [left_line], 0, (255,255,255), 2)
    cv.polylines(frame, [right_line], 0, (255,255,255), 2)

In [ ]:
def lane_detection(frame, flag=False):
    frame = cv.GaussianBlur(frame, (5, 5), 0)
    frame_hls = cv.cvtColor(frame, cv.COLOR_BGR2HLS)
    white_lower = (0, 180, 0)
    white_upper = (254, 254, 254)
    if flag:
        white_lower = (0, 180, 0)
        white_upper = (240, 240, 240)

    white_mask = cv.inRange(frame_hls, white_lower, white_upper)

    frame_lab = cv.cvtColor(frame, cv.COLOR_BGR2LAB)
    yellow_mask = cv.inRange(frame_lab, (150, 100, 140), (255, 140, 200))

    combined_mask = cv.bitwise_or(white_mask, yellow_mask)

    redImg = np.zeros(frame.shape, frame.dtype)
    redImg[:,:] = (0, 0, 255)
    redMask = cv.bitwise_and(redImg, redImg, mask=yellow_mask)

    greenImg = np.zeros(frame.shape, frame.dtype)
    greenImg[:,:] = (0, 255, 0)
    greenMask = cv.bitwise_and(greenImg, greenImg, mask=white_mask)

    masks = redMask + greenMask
    # if not flag:
    #     frame_luv = cv.cvtColor(frame, cv.COLOR_BGR2LUV)
    #     yellow_mask2 = cv.inRange(frame_luv, (150, 100, 150), (255, 140, 200))
    #     combined_yellow = cv.bitwise_and(yellow_mask, yellow_mask2)
    #     combined_mask = cv.bitwise_or(white_mask, combined_yellow)
    #     blueImg = np.zeros(frame.shape, frame.dtype)
    #     blueImg[:,:] = (255, 0, 0)
    #     blueMask = cv.bitwise_and(blueImg, blueImg, mask=yellow_mask2)
    #     masks = masks + blueMask

    kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (7, 3))
    lanes = cv.morphologyEx(frame_hls[:, :, 1], cv.MORPH_TOPHAT, kernel)

    kernel = cv.getStructuringElement(cv.MORPH_RECT, (13, 13))
    lanes_yellow = cv.morphologyEx(frame_lab[:, :, 2], cv.MORPH_TOPHAT, kernel)

    ret, lanes_yellow = cv.threshold(lanes_yellow, thresh=2, maxval=255, type=cv.THRESH_BINARY)
    lanes_yellow = cv.morphologyEx(lanes_yellow, cv.MORPH_OPEN, np.array([[0,1,0],[1,1,1],[0,1,0]], 'uint8'), iterations=4)

    lanes = cv.morphologyEx(frame_hls[:, :, 1], 
                              cv.MORPH_BLACKHAT,
                              kernel)

    ret, lanes = cv.threshold(lanes, thresh=5, maxval=255, type=cv.THRESH_BINARY)

    combined_mask = cv.bitwise_or(combined_mask, lanes_yellow)

    combined_mask = cv.morphologyEx(combined_mask, cv.MORPH_OPEN, np.array([[0,1,0],[1,1,1],[0,1,0]], 'uint8'), iterations=2)
    # combined_mask = cv.Canny(combined_mask, 50, 150)
    frame = cv.bitwise_and(frame, frame, mask=combined_mask)
    frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    
    cv.imshow("Masks", masks)
    cv.imshow("TOP_HAT_LANES_YELLOW", lanes_yellow)
    cv.imshow("BLACK_HAT_LANES", lanes)
    cv.imshow("Final", combined_mask)
    return frame

In [ ]:
def correct_Brightness(frame):
    frame_hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
    brightness = frame_hsv[...,2].mean()
    brightness_upper_limit = 115
    brightness_lower_limit = 85
    flag = False
    if brightness > brightness_upper_limit:
        hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
        h, s, v = cv.split(hsv)
        
        lim = round(0 + brightness - brightness_upper_limit)
        v[v < lim] = 0
        v[v >= lim] -= lim

        final_hsv = cv.merge((h, s, v))
        frame = cv.cvtColor(final_hsv, cv.COLOR_HSV2BGR)
        flag = True
    
    if brightness < brightness_lower_limit:
        hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
        h, s, v = cv.split(hsv)
        
        lim = round(255 + brightness_lower_limit - brightness)
        v[v > lim] = 255
        v[v <= lim] += lim

        final_hsv = cv.merge((h, s, v))
        frame = cv.cvtColor(final_hsv, cv.COLOR_HSV2BGR)
        flag = False

    return frame, flag, brightness

In [ ]:
def combine_two_color_images(image1, image2):
    foreground, background = image1.copy(), image2.copy()
    
    foreground_height = foreground.shape[0]
    foreground_width = foreground.shape[1]
    alpha =0.5
    
    # do composite on the upper-left corner of the background image.
    blended_portion = cv.addWeighted(foreground,
                alpha,
                background[:foreground_height,:foreground_width,:],
                1 - alpha,
                0,
                background)
    background[:foreground_height,:foreground_width,:] = blended_portion
    return background

# Main


In [ ]:
# Open video file
# video_file = "project"
video_file = "challenge"
# video_file = "harder_challenge"
capture = cv.VideoCapture("./img/Udacity/" + video_file + "_video.mp4")

# Check if camera opened successfully
if capture.isOpened() == False:
    print("Error opening video stream or file")

# Start timer for fps counter
start_timer = time.time() - 0.01
frame_count = -1

# Read every frame
while capture.isOpened():
    ret, frame = capture.read()

    # Check if there is another frame
    if frame is None:
        break

    orig = frame.copy()
    orig = undistort_image(orig)

    # Calculate Frame rate
    frame_count += 1
    ellapsed_time = time.time() - start_timer
    frame_rate = frame_count / ellapsed_time

    if ret == True:
        start_time_measurement("frame")

        start_time_measurement("Undistort")
        frame = undistort_image_remap(frame)
        end_time_measurement("Undistort")

        start_time_measurement("Warp")
        frame = warp_image_udacity(frame)
        end_time_measurement("Warp")

        start_time_measurement("Brightness correction")
        frame, flag, brightness = correct_Brightness(frame)
        end_time_measurement("Brightness correction")

        start_time_measurement("lane_detection")
        frame = lane_detection(frame, flag)
        end_time_measurement("lane_detection")

        start_time_measurement("Sliding windows")
        midpoint, lefts, rights = sliding_windows(frame, minimum_whites=margin, show_windows=True)
        end_time_measurement("Sliding windows")

        start_time_measurement("rewarp")
        frame = rewarp_image_udacity(frame)
        end_time_measurement("rewarp")

        y_values = np.linspace(len(img), 0, nwindows+1).astype(int)

        #--------------
        frame = cv.cvtColor(frame, cv.COLOR_GRAY2BGR)

        # start_time_measurement("Draw plane")
        # borderPoints = np.concatenate((lefts, np.flip(rights, axis=0)))
        # cv.drawContours(frame, [borderPoints], -1, (255,0,0), -1)
        # end_time_measurement("Draw plane")
        
        out = combine_two_color_images(frame, orig)
        #---------------

        # Add frame rate to video
        cv.putText(out, "FPS: " + str(round(frame_rate)), (0, 25),
                   cv.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2, cv.LINE_AA,)
        cv.putText(out, "Brightness: " + str(round(brightness)), (0, 50),
                   cv.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2, cv.LINE_AA,)
        cv.imshow("Frame", out)


        end_time_measurement("frame")
        # Close video with letter 'q'
        if cv.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        break

# When everything done, release the video capture object
capture.release()
cv.destroyAllWindows()

analyse_time_measurements()